In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import plotting

from matplotlib.pyplot import boxplot
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error 
from sklearn.model_selection import GridSearchCV


# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')


### Exploratory Data Analysis¶


In [12]:

df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
X_test = test_df.drop(columns=['ID'])


print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (5760, 15).




,ID,Date,y,Hour,Temperature(�C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(�C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [13]:
df.drop(columns='ID').drop_duplicates()


df[['Day','Month','Year']] = df.Date.str.split("/",expand=True)
df['Day'] = df['Day'].astype('int64')
df['Year'] = df['Year'].astype('int64')
df['Month'] = df['Month'].astype('int64')

df['New_Month'] = 0
df['New_Month'] = np.where(((df['Month']>=5) & (df['Month']<=10)),1,df['New_Month'])


import calendar
df['weekday'] = 0
for ind in df.index:
     df['weekday'][ind]= calendar.weekday(df['Year'][ind], df['Month'][ind] , df['Day'][ind])

X_test[['Day','Month','Year']] = X_test.Date.str.split("/",expand=True)
X_test['Day'] = X_test['Day'].astype('int64')
X_test['Year'] = X_test['Year'].astype('int64')
X_test['Month'] = X_test['Month'].astype('int64')

X_test['New_Month'] = 0
X_test['New_Month'] = np.where(((X_test['Month']>=5) & (X_test['Month']<=10)),1,X_test['New_Month'])


X_test['weekday'] = 0
for ind in X_test.index:
     X_test['weekday'][ind]= calendar.weekday(X_test['Year'][ind], X_test['Month'][ind] , X_test['Day'][ind])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
df =  pd.get_dummies(df , columns = ['Seasons'] , drop_first = True)

df = pd.get_dummies(df , columns = ['Holiday'], drop_first = True)

df = pd.get_dummies(df , columns = ['Functioning Day'], drop_first = True)

df['Temperature(�C)']=df['Temperature(�C)']+273
X_test['Temperature(�C)']=X_test['Temperature(�C)']+273


df['Dew point temperature(�C)']=df['Dew point temperature(�C)']+273
X_test['Dew point temperature(�C)']=X_test['Dew point temperature(�C)']+273


X_test =  pd.get_dummies(X_test , columns = ['Seasons'], drop_first = True)

X_test = pd.get_dummies(X_test, columns = ['Holiday'], drop_first = True)

X_test = pd.get_dummies(X_test, columns = ['Functioning Day'], drop_first = True)


df['Solar_Radiation']= np.log(df['Solar Radiation (MJ/m2)']+1)
X_test['Solar_Radiation']= np.log(X_test['Solar Radiation (MJ/m2)']+1)



df['is_night'] = 0
df.loc[(df['Hour'] < 6) | (df['Hour'] > 20), 'is_night'] = 1

X_test['is_night'] = 0
X_test.loc[(X_test['Hour'] < 6) | (X_test['Hour'] > 20), 'is_night'] = 1

df['HourRange']=0
np.where((((df['Hour'] <= 22) | (df['Hour'] >= 17))|(df['Hour']==8)),1,df['HourRange']) 



X_test['HourRange']=0
np.where((((X_test['Hour'] <= 22) | (X_test['Hour'] >= 17))|(X_test['Hour']==8)),1,X_test['HourRange']) 

df["weekend"] = df["weekday"].apply(lambda x: 1 if  x == 5 or x == 6 else 0 )
X_test["weekend"] = X_test["weekday"].apply(lambda x: 1 if  x == 5 or x == 6 else 0 )


conditions = [
    (df['Visibility (10m)'] >= 0) & (df['Visibility (10m)'] <=500),
    (df['Visibility (10m)'] > 500) & (df['Visibility (10m)'] <=1000),
    (df['Visibility (10m)'] > 1000) & (df['Visibility (10m)'] <=1500),
    (df['Visibility (10m)'] > 1500) & (df['Visibility (10m)'] <=2000)]
choices = [4 , 3 , 2 , 1]
df['level of risk'] = np.select(conditions, choices, default=0)


conditions1 = [
    (X_test['Visibility (10m)'] >= 0) & (X_test['Visibility (10m)'] <=500),
    (X_test['Visibility (10m)'] > 500) & (X_test['Visibility (10m)'] <=1000),
    (X_test['Visibility (10m)'] > 1000) & (X_test['Visibility (10m)'] <=1500),
    (X_test['Visibility (10m)'] > 1500) & (X_test['Visibility (10m)'] <=2000)]
choices = [4 , 3 , 2 , 1]
X_test['level of risk'] = np.select(conditions1, choices, default=0)


df["day_cos"] = np.cos((df['Hour']) * (2 * np.pi / 24))
df["day_sin"] = np.sin((df['Hour']) * (2 * np.pi / 24))
df["month_cos"] = np.cos((df['Month']) * (2 * np.pi / 12))
df["month_sin"] = np.sin((df['Month']) * (2 * np.pi / 12))

df["weekday_sin"] = np.sin((df['weekday']) * (2 * np.pi / 7))
df["weekday_cos"] = np.cos((df['weekday']) * (2 * np.pi / 7))



X_test["day_cos"] = np.cos((X_test['Hour']) * (2 * np.pi / 24))
X_test["day_sin"] = np.sin((X_test['Hour']) * (2 * np.pi / 24))
X_test["month_cos"] = np.cos((X_test['Month']) * (2 * np.pi / 12))
X_test["month_sin"] = np.sin((X_test['Month']) * (2 * np.pi / 12))
X_test["weekday_sin"] = np.sin((X_test['weekday']) * (2 * np.pi / 7))
X_test["weekday_cos"] = np.cos((X_test['weekday']) * (2 * np.pi / 7))





df['Rainfall(cm)'] = df['Rainfall(mm)'].apply(lambda x:x/10)
df['Rain&snow'] = df['Rainfall(cm)'] * df['Snowfall (cm)']
X_test['Rainfall(cm)'] = X_test['Rainfall(mm)'].apply(lambda x:x/10)
X_test['Rain&snow'] = X_test['Rainfall(cm)'] * X_test['Snowfall (cm)']





In [15]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'y'])
y_train = train_df['y']

X_val = val_df.drop(columns=['ID', 'y'])
y_val = val_df['y']


In [16]:
X_train = X_train [['level of risk','weekday_sin','weekday_cos','day_cos','day_sin','month_cos','month_sin','Month','is_night','weekday','Hour','Temperature(�C)','Humidity(%)', 'Solar_Radiation'  , 'Seasons_Summer','Seasons_Winter','Seasons_Spring' ,'Holiday_No Holiday' , 'Functioning Day_Yes' ,'New_Month','Wind speed (m/s)','Rainfall(mm)','Visibility (10m)']]
X_val = X_val [list(X_train.columns)]
X_test = X_test[list(X_train.columns)]


In [17]:
RegModel=XGBRegressor(learning_rate = 0.07,
                max_depth = 10, alpha = 6, n_estimators = 350)
    
y_train = np.log(train_df['y']+1)
#y_al = np.log(val_df['y']+1)


#Fit the trained model
RegModel.fit(X_train,y_train)

predict= RegModel.predict(X_val)
predict=np.exp(predict)

train_predict=RegModel.predict(X_train)
train_predict=np.exp(train_predict)
Error1= np.sqrt(mean_squared_log_error (np.exp(y_train) , train_predict ))
print('RMSLE Training:' + str(Error1))


predict= RegModel.predict(X_val)
predict=np.exp(predict)
Error= np.sqrt(mean_squared_log_error (y_val , predict ))
print('RMSLE Validation:' + str(Error))



[10:20:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSLE Training:0.016775914104677835
RMSLE Validation:0.3254471172574085


In [18]:
y_test_predicted = RegModel.predict(X_test)


y_test_predicted=np.exp(y_test_predicted)

In [19]:
# You should update/remove the next line once you change the features used for trainin

test_df['y'] = y_test_predicted.astype(int)



In [20]:
test_df[['ID', 'y']].to_csv('submission.csv', index=False)
